In [1]:
import numpy as np
import pandas as pd

In [2]:
sub_path = './submit/'

res_pre_1 = pd.read_csv(sub_path + 'lgb_one.csv', encoding='utf-8')
res_pre_2 = pd.read_csv(sub_path + 'lgb_two.csv', encoding='utf-8')
res_pre_3 = pd.read_csv(sub_path + 'chusai_lstm_model_1.csv', encoding='utf-8')
res_pre_4 = pd.read_csv(sub_path + 'chusai_lstm_model_2.csv', encoding='utf-8')

res1 = res_pre_1[['id']]
res1['forecastVolum'] = list(map(lambda x, y: int((x + y) / 2), list(res_pre_1['forecastVolum']), list(res_pre_2['forecastVolum'])))

res2 = pd.read_csv(sub_path + 'rule.csv', encoding='utf-8')

res3 = res_pre_3[['id']]
res3['forecastVolum'] = list(map(lambda x, y: int((x + y) / 2), list(res_pre_3['forecastVolum']), list(res_pre_4['forecastVolum'])))

res4 = pd.read_csv(sub_path + 'sub_method_one.csv', encoding='utf-8')

test = pd.read_csv('./data/evaluation_public.csv', encoding='utf-8')[['id', 'regMonth']]

res1.columns = ['id', 'p1']
res2.columns = ['id', 'p2']
res3.columns = ['id', 'p3']
res4.columns = ['id', 'p4']

res = pd.merge(res1, res2, on='id', how='outer')
res = pd.merge(res, res3, on='id', how='outer')
res = pd.merge(res, res4, on='id', how='outer')
res = pd.merge(test, res, on='id', how='outer')

print(res.groupby(['regMonth'], as_index=False)['p1'].mean())
print(res.groupby(['regMonth'], as_index=False)['p2'].mean())
print(res.groupby(['regMonth'], as_index=False)['p3'].mean())
print(res.groupby(['regMonth'], as_index=False)['p4'].mean())


w11, w21, w31, w41 = 0.2, 0.2, 0.2, 0.4
w12, w22, w32, w42 = 0.25, 0.25, 0.2, 0.3
w13, w23, w33, w43 = 0.33, 0.0, 0.34, 0.33
w14, w24, w34, w44 = 0.33, 0.0, 0.34, 0.33


def fusion(value_1, value_2, value_3, value_4, month):
    if month == 1:
        return value_1 * w11 + value_2 * w21 + value_3 * w31 + value_4 * w41
    elif month == 2:
        return value_1 * w12 + value_2 * w22 + value_3 * w32 + value_4 * w42
    elif month == 3:
        return value_1 * w13 + value_2 * w23 + value_3 * w33 + value_4 * w43
    elif month == 4:
        return value_1 * w14 + value_2 * w24 + value_3 * w34 + value_4 * w44


res['forecastVolum'] = res.apply(lambda index: fusion(index.p1, index.p2, index.p3, index.p4, index.regMonth), axis=1)
res['forecastVolum'] = res['forecastVolum'].map(lambda index: int(np.round(index)))
print(res[['p1', 'p2', 'p3', 'p4']].corr())
print(res.describe())
print(res.groupby(['regMonth'], as_index=False)['forecastVolum'].mean())
print(res['forecastVolum'].mean())
res[['id', 'forecastVolum']].to_csv('./submit/fusion_chusai.csv', encoding='utf-8', index=False)

   regMonth          p1
0         1  475.275000
1         2  321.031061
2         3  488.022727
3         4  478.614394
   regMonth          p2
0         1  510.012879
1         2  331.528788
2         3  472.425758
3         4  469.428788
   regMonth          p3
0         1  536.573485
1         2  424.113636
2         3  389.911364
3         4  389.925000
   regMonth          p4
0         1  559.806061
1         2  374.103788
2         3  474.549242
3         4  482.976515
          p1        p2        p3        p4
p1  1.000000  0.965556  0.915804  0.950894
p2  0.965556  1.000000  0.937084  0.963790
p3  0.915804  0.937084  1.000000  0.961396
p4  0.950894  0.963790  0.961396  1.000000
                id    regMonth           p1           p2           p3  \
count  5280.000000  5280.00000  5280.000000  5280.000000  5280.000000   
mean   2677.166667     2.50000   440.735795   445.849053   435.130871   
std    1549.503211     1.11814   580.418600   546.839464   540.912516   
min       1.0